In [1]:
from IPython.display import display
from config.mapa import (
    crear_mapa,
    crear_capa,
    agregar_marcadores,
    agregar_geojson,
    agregar_control_capas,
    agregar_mapa_calor,
    agregar_color_seguridad
)
from config.datos import cargar_datos, filtrar_geojson, cargar_geojson, calcular_porcentaje_robos, contar_robos_por_distrito, guardar_porcentaje_robos_csv
from helper import distritos

def main():
    mapa = crear_mapa()
    geojson_data = cargar_geojson("data/distritos-peru.geojson")
    datos_incidentes = cargar_datos("./data/noticias-datos-manuales.json")
    
    robos_por_distrito = contar_robos_por_distrito(datos_incidentes)
    df_porcentaje_robos = calcular_porcentaje_robos(robos_por_distrito)
    df_porcentaje_robos['Distrito'] = df_porcentaje_robos['Distrito'].str.lower()
    guardar_porcentaje_robos_csv(df_porcentaje_robos, "./data/porcentaje_robos.csv")
    
    dict_porcentaje_robos = df_porcentaje_robos.set_index('Distrito')['Porcentaje de robos'].to_dict()

    def style_function(feature):
        distrito = feature['properties']['nombdist'].lower()
        porcentaje_robos = dict_porcentaje_robos.get(distrito)
        
        if porcentaje_robos is None:
            color = 'grey'
        elif porcentaje_robos > 50:
            color = 'red'  # Inseguro
        elif porcentaje_robos > 20:
            color = 'orange'  # Moderadamente seguro
        else:
            color = 'green'  # Seguro

        return {
            'fillColor': color,
            'fillOpacity': 0.5,
            'weight': 2,
            'color': 'black',
        }

    capa_marcadores = crear_capa("Incidencias")
    capa_geojson = crear_capa("Distritos de Ica")
    capa_mapa_calor = crear_capa("Mapa de Calor")
    capa_color_seguridad = crear_capa("Color de Seguridad")
    
    agregar_marcadores(mapa, capa_marcadores, datos_incidentes)
    agregar_mapa_calor(mapa, datos_incidentes, capa_mapa_calor)
    agregar_geojson(capa_geojson, geojson_data)
    agregar_color_seguridad(capa_color_seguridad, geojson_data, style_function)
    
    geojson_data = filtrar_geojson(geojson_data, distritos.distritos_ica)
    mapa.add_child(capa_marcadores)
    mapa.add_child(capa_geojson)
    mapa.add_child(capa_mapa_calor)
    mapa.add_child(capa_color_seguridad)
    agregar_control_capas(mapa)
    display(mapa)


if __name__ == "__main__":
    main()